## 📌 Kotlin에서 Java 호출 시 주의할 점

### 1. Java 클래스 사용

* Kotlin에서 Java 클래스를 가져다 쓰는 건 특별한 설정 없이 가능.
* `import`만 자동으로 추가되며, 코드만 보면 Kotlin 클래스인지 Java 클래스인지 구분하기 어려움.

```java
// Java 코드
public class Car {
    private String color;
    private String model;
    private int year;

    public Car(String color, String model, int year) {
        this.color = color;
        this.model = model;
        this.year = year;
    }

    // getter / setter
}
```

```kotlin
// Kotlin 코드
val car = Car("blue", "Ford", 2015)
println(car)
```

👉 Java 클래스도 Kotlin 코드에서 일반적인 클래스처럼 사용 가능.

---

### 2. Null 안정성 (Null-safety)

* Kotlin은 `null` 안전을 보장하지만, Java는 보장하지 않음.
* 따라서 Kotlin 컴파일러는 Java 코드를 가져올 때 **애너테이션**을 힌트로 활용.

#### (1) `@NotNull`

* `@NotNull` 붙은 매개변수 → Kotlin에서 **non-null 타입**으로 취급.
* Kotlin 컴파일러가 **런타임 체크 코드**를 자동 생성해줌.
* 만약 null을 넣으면 **IllegalArgumentException** 발생.

```java
public void setColor(@NotNull String color) {
    this.color = color;
}
```

```kotlin
car.color = null // 런타임 예외 발생
```

---

#### (2) `@Nullable`

* `@Nullable` 붙은 경우 → Kotlin에서 **nullable 타입**(`String?`)으로 매핑.
* null을 넣어도 예외 발생하지 않음.

```java
public void setColor(@Nullable String color) {
    this.color = color;
}
```

```kotlin
car.color = null // 예외 없음
```

---

#### (3) 애너테이션 없는 경우 → **플랫폼 타입 (Platform Type)**

* Kotlin 컴파일러가 null 여부를 알 수 없을 때 생성됨.
* `String!` 같은 형태로 표시됨 (IDE에서 확인 가능).
* 개발자가 nullable/nonnull로 **직접 선택**해 사용해야 함.

```kotlin
val model = car.model  // String! (플랫폼 타입)

// nullable로 받을 수도 있고
val m1: String? = car.model

// non-nullable로 받을 수도 있음
val m2: String = car.model
```

⚠️ 단, non-nullable로 받았는데 실제 값이 `null`이면 NPE가 발생할 수 있음.
👉 따라서 **Java 코드에 @NotNull/@Nullable을 붙이는 게 중요**.

---


| 구분                            | Kotlin 코드                                                      | Java 호출 방식                                        | 비고                                 |
| ----------------------------- | -------------------------------------------------------------- | ------------------------------------------------- | ---------------------------------- |
| **Top-level 함수**              | `fun topLevel() { ... }`                                       | `CarKt.topLevel();`                               | Kotlin 파일명 + `Kt` 클래스 생성           |
| **Top-level 함수 (JVMName 변경)** | `@file:JvmName("StaticCar") fun topLevel() { ... }`            | `StaticCar.topLevel();`                           | 이름 충돌 방지 가능                        |
| **Extension 함수**              | `fun String.printMe() { println(this) }`                       | `CarKt.printMe("Hello");`                         | Java에서는 static 메서드처럼 호출            |
| **val 프로퍼티**                  | `val model: String`                                            | `car.getModel();`                                 | getter만 생성, setter 없음              |
| **var 프로퍼티**                  | `var color: String`                                            | `car.getColor();` / `car.setColor("red");`        | 기본 getter/setter 사용 가능             |
| **Custom getter/setter**      | `var color: String get() = field set(value) { field = value }` | Java에서 getter/setter 그대로 호출                       | 커스텀 로직 포함 가능                       |
| **Boolean 프로퍼티**              | `var isAutomatic: Boolean`                                     | `car.isAutomatic();` / `car.setAutomatic(false);` | getter는 `is`, setter는 `set` 접두사 사용 |
| **@JvmField 적용**              | `@JvmField var model: String`                                  | `car.model` / `car.model = "Audi";`               | getter/setter 없이 직접 접근 가능          |
| **private set**               | `var color: String private set`                                | Java에서 `car.setColor(...)` 접근 불가                  | setter 접근 제한                       |
| **val + private set**         | `val color: String private set`                                | Java에서 `car.setColor(...)` 접근 불가, getter만 가능      | 읽기 전용                              |
| **프로퍼티 이름 CamelCase**         | `var modelName: String`                                        | `car.getModelName();`                             | CamelCase 유지                       |
